In [13]:
import time, urllib.request, multiprocessing as mp, concurrent.futures

In [2]:
def seq_download_images(img_numbers):
    total_bytes = 0
    for num in img_numbers:
        total_bytes += _download_image(num)
    return total_bytes

In [16]:
def _download_image(image_number):
    print(image_number)
    image_url = 'http://699340.youcanlearnit.net/image{:03d}.jpg'.format(image_number)
    try:
        with urllib.request.urlopen(image_url, timeout=69) as conn:
            return len(conn.read())
    except urllib.error.HTTPError:
        print('Oh god', image_number)
    except Exception as e:
        print(e)

In [15]:
def par_download_images(image_numbers):
    total_bytes = 0
    with concurrent.futures.ThreadPoolExecutor() as pool:
        futures = [pool.submit(_download_image, num) for num in image_numbers]
        for f in concurrent.futures.as_completed(futures):
            total_bytes += f.result()
    return total_bytes

In [17]:
if __name__ == '__main__':
    NUM_EVAL_RUNS = 3
    IMAGE_NUMBERS = list(range(1,4))
    sequential_result = seq_download_images(IMAGE_NUMBERS)
    sequential_time = 0
    for i in range(NUM_EVAL_RUNS):
        print(i)
        start = time.perf_counter()
        seq_download_images(IMAGE_NUMBERS)
        sequential_time += time.perf_counter() - start
    sequential_time /= NUM_EVAL_RUNS
    parallel_result = par_download_images(IMAGE_NUMBERS)
    parallel_time = 0
    for i in range(NUM_EVAL_RUNS):
        start = time.perf_counter()
        par_download_images(IMAGE_NUMBERS)
        parallel_time += time.perf_counter() - start
    parallel_time /= NUM_EVAL_RUNS
    if sequential_result != parallel_result:
        raise Exception ('sequential_result and parallel_result do not match.')
    print('Average Sequential Time: {:.2f} ms'.format(sequential_time*1000))
    print('Average Parallel Time: {:.2f} ms'.format(parallel_time*1000))
    print('Speedup: {:.2f}'.format(sequential_time/parallel_time))
    print('Efficiency: {:.2f}%'.format(100*(sequential_time/parallel_time)/mp.cpu_count()))

1
2
3
0
1
2
3
1
1
2
3
2
1
2
3
12

3
1
2
3
12

3
1
2
3
Average Sequential Time: 2878.64 ms
Average Parallel Time: 955.16 ms
Speedup: 3.01
Efficiency: 37.67%
